In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os

from keras_preprocessing.image import load_img, img_to_array

%matplotlib inline

In [ ]:
src_dir = 'Dataset_PA/rps-cv-images'

train_dir = os.path.join('Dataset_PA', 'train')
val_dir = os.path.join('Dataset_PA', 'val')

paper_dir = os.path.join(src_dir,'paper')
rock_dir = os.path.join(src_dir,'rock')
scissors_dir = os.path.join(src_dir,'scissors')

In [ ]:
print('paper : ',len(os.listdir(paper_dir)))
print('rock : ',len(os.listdir(rock_dir)))
print('scissors : ',len(os.listdir(scissors_dir)))
print('Train : ',len(os.listdir(train_dir)))

paper :  712
rock :  726
scissors :  750
Train :  3


## Preprocessing

In [ ]:
train_generator = ImageDataGenerator(validation_split=0.4,
                               rescale=1/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               rotation_range=45,
                               horizontal_flip= True,
                               vertical_flip= True,
                               fill_mode='nearest')

val_generator=ImageDataGenerator(
    rescale=1/255
)

train_data = train_generator.flow_from_directory(train_dir,
                                           class_mode= 'categorical',
                                           shuffle = True,
                                           batch_size=32,
                                           target_size=(150,150))

val_data = val_generator.flow_from_directory(val_dir,
                                         class_mode= 'categorical',
                                         shuffle = True,
                                         batch_size=32,
                                         target_size=(150,150))

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.
